# Próba wykorzystania istniejącego modelu RegGNN
https://github.com/basiralab/RegGNN/blob/main/proposed_method/RegGNN.py

Nasze zadanie opiera się na przygotowaniu modelu rozwiązującego zadanie regresji grafowej, czyli na podstawie grafów, ich cech globalnych i etykiet chcemy prognozować wartość metryki średniej ilości wykorzystanych transceiverów podczas symulacji.

In [1]:
import torch

# Wczytaj zbiór uczący
train_dataset = torch.load('train_dataset.pt')
# wczytaj zbiór testowy
test_dataset = torch.load('test_dataset.pt')

print(len(train_dataset))
print(len(test_dataset))

184
46


# Przykładowy model regresyjny Grafowej sieci neuronowej GNN

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GraphConv

class GraphRegressionModel(nn.Module):
    def __init__(self, num_node_features=1, hidden_dim=64, output_dim=1, dropout=0.5):
        super(GraphRegressionModel, self).__init__()

        # Graph Convolutional Layer
        self.conv1 = GraphConv(num_node_features, hidden_dim)

        # Fully Connected Layers with Dropout
        self.fc1 = nn.Linear(hidden_dim + 2, hidden_dim)
        self.dropout = nn.Dropout(p=dropout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, edge_weight, global_feature = data.x, data.edge_index, data.edge_weight, data.global_feature

        # Apply Graph Convolution
        x = self.conv1(x, edge_index, edge_weight=edge_weight)

        # Global features concatenation
        global_feature = global_feature.expand(x.size(0), -1)  # Dostosuj global_feature do rozmiaru x
        x = torch.cat([x, global_feature], dim=1)

        # Fully Connected Layers with Dropout
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)

        return x
    
    def loss(self, pred, score):
        return F.mse_loss(pred, score)

In [3]:
from hyperopt import fmin, tpe, hp

# Zdefiniuj funkcję oceny (score function)
def objective(params):
    hidden_dim = int(params['hidden_dim'])
    dropout = params['dropout']

    model = GraphRegressionModel(hidden_dim=hidden_dim, dropout=dropout)
    
    # Definiuj optymalizator, liczbę epok, itp.
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    num_epochs = 200
    
    for epoch in range(num_epochs):
        # Trening i ewaluacja modelu
        for data in train_loader:
            optimizer.zero_grad()
            output = model(data)
            loss = model.loss(output, data.y.view(-1, 1).float())
            loss.backward()
            optimizer.step()
    
    # Ewaluacja na zbiorze testowym
    test_loss = 0.0  
    for data in test_loader:
        output = model(data)
        test_loss += model.loss(output, data.y.view(-1, 1).float()).item()
    
    # Zwróć funkcję oceny (score)
    return test_loss

# Przestrzeń poszukiwań dla hyperopt
space = {
    'hidden_dim': hp.quniform('hidden_dim', 32, 256, 32),  # Przeszukuj wartości co 32
    'dropout': hp.uniform('dropout', 0.05, 0.8),
}

# Minimalizacja funkcji oceny za pomocą algorytmu TPE
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10)  # Dla przykładu ustawiono max_evals na 10
print("Najlepsze hiperparametry:", best)

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]



KeyboardInterrupt



In [15]:
import torch
from torch_geometric.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)
# Ustawienie optymalnych hiperparametrów
#optimal_hidden_dim = int(best['hidden_dim'])
#optimal_dropout = best['dropout']

optimal_hidden_dim = 192
optimal_dropout = 0.06

# Inicjalizacja modelu z optymalnymi hiperparametrami
optimal_model = GraphRegressionModel(hidden_dim=optimal_hidden_dim, dropout=optimal_dropout)

# Definiuj DataLoader dla zbiorów uczącego i testowego
train_loader = DataLoader(train_dataset, batch_size=1 , shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

# Definiuj optymalizator, liczbę epok, itp.
optimizer = torch.optim.Adam(optimal_model.parameters(), lr=0.001)
num_epochs = 200

# Listy do śledzenia train loss i test loss
train_losses = []
test_losses = []

# Trening modelu
for epoch in range(num_epochs):
    if epoch % 10 == 0:
        print("Epoch:", epoch)
    for data in train_loader:
        optimizer.zero_grad()
        output = optimal_model(data)
        loss = optimal_model.loss(output, data.y.view(-1, 1).float())
        loss.backward()
        optimizer.step()

sample_data = test_dataset[0]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

cpu
Epoch: 0


C:\Users\jderd\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
C:\Users\jderd\AppData\Local\Temp\ipykernel_28284\794946686.py:36: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([28, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(pred, score)


Epoch: 10
Epoch: 20
Epoch: 30
Epoch: 40
Epoch: 50
Epoch: 60
Epoch: 70
Epoch: 80
Epoch: 90
Epoch: 100
Epoch: 110
Epoch: 120
Epoch: 130
Epoch: 140
Epoch: 150
Epoch: 160
Epoch: 170
Epoch: 180
Epoch: 190
Przewidywana wartość: 835.7713012695312
Rzeczywista wartość: 874.5499877929688


In [4]:
sample_data = test_dataset[9]

# Przeprowadzenie predykcji na przykładowym obiekcie
optimal_model.eval()
with torch.no_grad():
    prediction = optimal_model(sample_data)[0][0].item()

# Wyświetlenie wyników
print("Przewidywana wartość:", prediction)
print("Rzeczywista wartość:", sample_data.y.item())

Przewidywana wartość: 279.5835876464844
Rzeczywista wartość: 286.3299865722656


In [14]:
optimal_model.load_state_dict(torch.load("model"))

total_accuracy = 0.0

# Iteracja przez cały zbiór testowy
for i, sample_data in enumerate(test_dataset):
    # Przeprowadzenie predykcji na przykładowym obiekcie
    optimal_model.eval()
    with torch.no_grad():
        prediction = optimal_model(sample_data)[0][0].item()
    
    # Wyświetlenie wyników
    print(f"Przykład {i + 1}:")
    print("Przewidywana wartość:", prediction)
    print("Rzeczywista wartość:", sample_data.y.item())
    
    # Obliczenie procentowej jakości regresji
    actual_value = sample_data.y.item()
    accuracy = 100 * (1 - abs(prediction - actual_value) / actual_value)
    total_accuracy += accuracy
    
    print(f"Procentowa jakość regresji: {accuracy:.2f}%\n")

# Obliczenie średniej jakości procentowej
average_accuracy = total_accuracy / len(test_dataset)
print(f"Średnia procentowa jakość regresji dla wszystkich przykładów: {average_accuracy:.2f}%")


Przykład 1:
Przewidywana wartość: 830.4153442382812
Rzeczywista wartość: 874.5499877929688
Procentowa jakość regresji: 94.95%

Przykład 2:
Przewidywana wartość: 578.1353759765625
Rzeczywista wartość: 569.8499755859375
Procentowa jakość regresji: 98.55%

Przykład 3:
Przewidywana wartość: 458.70751953125
Rzeczywista wartość: 450.54998779296875
Procentowa jakość regresji: 98.19%

Przykład 4:
Przewidywana wartość: 876.1487426757812
Rzeczywista wartość: 839.4600219726562
Procentowa jakość regresji: 95.63%

Przykład 5:
Przewidywana wartość: 704.1048583984375
Rzeczywista wartość: 705.010009765625
Procentowa jakość regresji: 99.87%

Przykład 6:
Przewidywana wartość: 689.338134765625
Rzeczywista wartość: 697.9400024414062
Procentowa jakość regresji: 98.77%

Przykład 7:
Przewidywana wartość: 865.16943359375
Rzeczywista wartość: 854.9099731445312
Procentowa jakość regresji: 98.80%

Przykład 8:
Przewidywana wartość: 797.7269287109375
Rzeczywista wartość: 813.02001953125
Procentowa jakość regresji:

In [8]:
torch.save(optimal_model.state_dict(), "model")

# Podsumowanie
Metryki:
R^2 i MSE
I w walidacji krzyżowej